## Access Google drive directory

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

folder_path = '/content/drive/Shareddrives/PediatricAppleWatchStudy/ECGDataPipelineTemplate'
os.chdir(folder_path)

!pip install fhir.resources
!pip install -i https://test.pypi.org/simple/ spezi-data-pipeline==0.1.0b1

## Import Spezi Data Pipeline

In [1]:
from spezi_data_pipeline.data_access.firebase_fhir_data_access import FirebaseFHIRAccess
from spezi_data_pipeline.data_flattening.fhir_resources_flattener import flatten_fhir_resources, FHIRDataFrame, ColumnNames
from modules.visualization import ECGDataViewer
from modules.utils import process_ecg_data, export_database_in_csv

### Download the ECG data

In [2]:
# Define the project's credentials
project_id = "cs342-2023-paws"
service_account_key_file = "Service account key/cs342-2023-paws-firebase-adminsdk-cz0fu-444bf65586.json"

# Connect to Firestore
firebase_access = FirebaseFHIRAccess(project_id, service_account_key_file)
firebase_access.connect()

# Download and process data
fhir_observations = firebase_access.fetch_data("users", "HealthKit", ["131328"])
flattened_fhir_dataframe = flatten_fhir_resources(fhir_observations)

ecg_data = process_ecg_data(firebase_access.db, flattened_fhir_dataframe.df)

### Preview the ECG data

In [4]:
ecg_data.head()

,UserId,GenderIdentityKey,DateOfBirthKey,ResourceId,EffectiveDateTime,NumberOfMeasurements,SamplingFrequency,SamplingFrequencyUnit,ElectrocardiogramClassification,HeartRate,...,Diagnosis3_tracingQuality,Diagnosis2_physicianInitials,Diagnosis1_diagnosisDate,Diagnosis3_notes,Diagnosis2_physicianDiagnosis,Diagnosis1_physicianInitials,ECGDataRecording1,ECGDataRecording2,ECGDataRecording3,AgeGroup
29,k3BnzOGAO0fIaxkDVXTZKlj3LAu2,male,2008-03-28 04:42:00+00:00,50D5D295-DCC6-4939-A0F2-526C3587FA58,2024-05-05,15383.0,512.773438,hertz,inconclusiveOther,66.0,...,None,None,2024-05-16 01:29,None,None,VB,"[0.23351129150390626, 0.3176947326660156, 0.37...","[-0.045341487884521484, -0.04632181549072266, ...","[-0.09912637329101562, -0.10237238311767578, -...",Child
0,3EUoHxIuYkWMKcnLfK38nTGOqHn1,female,1993-10-03 19:16:00+00:00,09F93FED-99F2-40DF-9AB9-DF1909E8D0C5,2023-03-13,15360.0,512.000000,hertz,sinusRhythm,108.0,...,Good,VB,2024-04-15 20:13,Test notes.,Normal,VB,"[-0.10455480194091797, -0.09996173858642578, -...","[-0.04079214096069336, -0.04222143173217773, -...","[0.005091599941253662, 0.0034010891914367674, ...",Adult
1,3EUoHxIuYkWMKcnLfK38nTGOqHn1,female,1993-10-03 19:16:00+00:00,0FCBF788-6B4C-41C5-A138-AB2BC1E5B8BD,2023-03-15,15360.0,512.000000,hertz,sinusRhythm,81.0,...,Uninterpretable,SC,2024-04-15 20:14,,Normal,TB,"[-0.10653959655761719, -0.11071208953857421, -...","[0.12884559631347656, 0.12435533905029297, 0.1...","[-0.09334623718261718, -0.09919612884521484, -...",Adult
28,k3BnzOGAO0fIaxkDVXTZKlj3LAu2,male,2008-03-28 04:42:00+00:00,2641A3E9-0D9D-405A-8CA8-731975D2FA05,2024-03-28,15383.0,512.773438,hertz,sinusRhythm,56.0,...,None,None,2024-05-01 18:35,None,None,SC,"[-0.003155410051345825, -0.0021231422424316405...","[-0.0436302490234375, -0.04745901489257812, -0...","[-0.07754312133789062, -0.07865031433105468, -...",Child
30,k3BnzOGAO0fIaxkDVXTZKlj3LAu2,male,2008-03-28 04:42:00+00:00,A1A44124-8F59-476D-AA13-369BA4CD0EF1,2024-03-28,15383.0,512.773438,hertz,sinusRhythm,57.0,...,Good,AZ,2024-04-30 22:01,,Normal,AZ,"[0.08295372009277344, 0.0893570556640625, 0.09...","[0.15599002075195312, 0.15335098266601563, 0.1...","[0.044361488342285156, 0.044864055633544925, 0...",Child


### Export the database in .CSV format

In [5]:
export_database_in_csv(ecg_data)

## Review ECG recordings

In [4]:
viewer = ECGDataViewer(ecg_data, firebase_access.db)

Dropdown(description='Your Initials:', options=('Select', 'AZ', 'SC', 'TB', 'VB', 'Other'), value='Select')

Textarea(value='', description='Your Initials:', layout=Layout(visibility='hidden', width='300px'), placeholde…

HTML(value='')

Output()

Output()

Output()

Button(button_style='info', description='LOAD MORE', icon='plus', layout=Layout(height='50px', width='200px'),…